In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
sys.path.append('/Users/Tim/PycharmProjects/HOI/')

In [12]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from ipywidgets import IntProgress
import itertools

import matplotlib.pyplot as plt
from numpy import sign, sin, cos, pi
from numpy.random import normal, randn

In [13]:
from synthetic_data import stationary_pb_ts, nonstationary_ts_n, nonstat_egs
from HOI.preprocessings import compute_kernel, compute_kernel_n
from HOI.tests import test_independence
from HOI.statistics import compute_dHSIC_statistics

In [23]:
def stationary_ts_n(n_sample, t_time, d, mode, a=0.5):
    """
    Returns nonstationary time series data that has higher-order interactions
    """
    # variables * time * n_sample
    x_mat = []
    y_mat = []
    z_mat = []
    for j in range(n_sample):
        # np.random.seed(seed)
        x = np.zeros(t_time)
        y = np.zeros(t_time)
        z = np.zeros(t_time)

        x[0] = randn()
        y[0] = randn()
        z[0] = randn()
        for i in range(1, t_time):
            x[i] = a * x[i - 1] + randn()
            y[i] = a * y[i - 1] + randn()
            if mode == 'case1':
                # pairwise independent but jointly dependent
                z[i] = a * z[i - 1] + d * abs(randn()) * sign(x[i] * y[i]) + randn()
            if mode == 'case2':
                # 2 pairwise dependent and 3-way jointly dependent
                z[i] = a * z[i - 1] + d * (x[i] + y[i]) + randn()
            if mode == 'case3':
                # all independence
                z[i] = a * z[i - 1] + randn()

        x_mat.append(x)
        y_mat.append(y)
        z_mat.append(z)
    return np.array(x_mat), np.array(y_mat), np.array(z_mat)

# permute

In [77]:
power ={}
for d in tqdm(np.arange(0.1, 1, 0.1)):
    rejects = 0
    for i in tqdm(range(100)):
        d1, d2, d3 = stationary_ts_n(n_sample=20, t_time=50, d=d, mode='case2', a=0.5)
        kd1 = compute_kernel_n(d1)
        kd2 = compute_kernel_n(d2)
        kd3 = compute_kernel_n(d3)
        _, _, _, reject = test_independence([kd1, kd2, kd3], None, n_perms = 1000, alpha=0.05, mode = 'permutation')
        rejects = rejects + reject
    power[str(d)] = rejects/100
print(power)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{'0.1': 0.09, '0.2': 0.44, '0.30000000000000004': 0.81, '0.4': 0.96, '0.5': 0.99, '0.6': 1.0, '0.7000000000000001': 1.0, '0.8': 1.0, '0.9': 1.0}


# shift

In [74]:
power ={}
for d in tqdm(np.arange(0.1, 1, 0.1)):
    rejects = 0
    for i in tqdm(np.arange(100)):
        df = stationary_pb_ts(t_time = 50, d = d, mode = "case2", a=0.5)
        data_dict, kernel_dict = compute_kernel(df)
        _, _, _, reject = test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']],
                                 [data_dict['d1'], data_dict['d2'], data_dict['d3']],
                                 n_perms=1000, alpha=0.05, mode = 'shifting')
        rejects = rejects + reject
    power[str(d)] = rejects/100
    print(power)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{'0.1': 0.15}


  0%|          | 0/100 [00:00<?, ?it/s]

{'0.1': 0.15, '0.2': 0.15}


...

{'0.1': 0.15, '0.2': 0.15, '0.30000000000000004': 0.37, '0.4': 0.63, '0.5': 0.74, '0.6': 0.84, '0.7000000000000001': 0.91}


  0%|          | 0/100 [00:00<?, ?it/s]

{'0.1': 0.15, '0.2': 0.15, '0.30000000000000004': 0.37, '0.4': 0.63, '0.5': 0.74, '0.6': 0.84, '0.7000000000000001': 0.91, '0.8': 0.96}


  0%|          | 0/100 [00:00<?, ?it/s]

{'0.1': 0.15, '0.2': 0.15, '0.30000000000000004': 0.37, '0.4': 0.63, '0.5': 0.74, '0.6': 0.84, '0.7000000000000001': 0.91, '0.8': 0.96, '0.9': 0.99}


In [14]:
def almost_iid(n_sample, t_time, d, mode, a=0.5):
    """
    Returns nonstationary time series data that has higher-order interactions
    """
    # variables * time * n_sample
    x_mat = []
    y_mat = []
    z_mat = []
    for j in range(n_sample):
        # np.random.seed(seed)
        x = np.zeros(t_time)
        y = np.zeros(t_time)
        z = np.zeros(t_time)

        x[0] = randn()
        y[0] = randn()
        z[0] = randn()
        for i in range(1, t_time):
            x[i] = a * x[i - 1] + randn()
            y[i] = a * y[i - 1] + randn()
            if mode == 'case1':
                # pairwise independent but jointly dependent
                z[i] = a * z[i - 1] + d * abs(randn()) * sign(x[i] * y[i]) + randn()
            if mode == 'case2':
                # 2 pairwise dependent and 3-way jointly dependent
                z[i] = a * z[i - 1] + d * (x[i] + y[i]) + randn()
            if mode == 'case3':
                # all independence
                z[i] = a * z[i - 1] + randn()

        x_mat.append(x)
        y_mat.append(y)
        z_mat.append(z)
  
    return np.array(x_mat*2), np.array(y_mat*2), np.array(z_mat*2)

In [30]:
power ={}
for d in tqdm(np.arange(0.01, 1, 0.2)):
    rejects = 0
    for i in tqdm(range(100)):
        d1, d2, d3 = almost_iid(n_sample=20, t_time=10, d=d, mode='case2', a=0.5)
        kd1 = compute_kernel_n(d1)
        kd2 = compute_kernel_n(d2)
        kd3 = compute_kernel_n(d3)
        _, _, _, reject = test_independence([kd1, kd2, kd3], None, n_perms = 1000, alpha=0.05, mode = 'permutation')
        rejects = rejects + reject
    power[str(d)] = rejects/100
    print(power)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{'0.01': 1.0}


  0%|          | 0/100 [00:00<?, ?it/s]

{'0.01': 1.0, '0.21000000000000002': 1.0}


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [28]:
power ={}
for d in tqdm(np.arange(0.01, 1, 0.2)):
    rejects = 0
    for i in tqdm(range(100)):
        d1, d2, d3 = stationary_ts_n(n_sample=40, t_time=10, d=d, mode='case2', a=0.5)
        kd1 = compute_kernel_n(d1)
        kd2 = compute_kernel_n(d2)
        kd3 = compute_kernel_n(d3)
        _, _, _, reject = test_independence([kd1, kd2, kd3], None, n_perms = 1000, alpha=0.05, mode = 'permutation')
        rejects = rejects + reject
    power[str(d)] = rejects/100
    print(power)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{'0.01': 0.02}


  0%|          | 0/100 [00:00<?, ?it/s]

{'0.01': 0.02, '0.21000000000000002': 0.62}


  0%|          | 0/100 [00:00<?, ?it/s]

{'0.01': 0.02, '0.21000000000000002': 0.62, '0.41000000000000003': 1.0}


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 